# Tutorial 1: Matching

## 0. Import libraries 

In [69]:
import pandas as pd
import numpy as np
from hypex import Matcher
import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [70]:
from hypex.dataset.dataset import Dataset

In [86]:
data = Dataset(na_columns=['feature_col_3', 'feature_col_2'], 
               num_main_causes_cols=2)
data.df

,info_col_1,info_col_2,feature_col_1,feature_col_2,feature_col_3,feature_col_4,treatment_1,outcome_1
0,1504,G,female,Investment,NaN,0.0,0.0,-0.206974
1,5044,G,male,NaN,0.764466,0.0,1.0,1.510945
2,7360,E,female,Deposit,0.036796,0.0,1.0,1.392190
3,14971,G,male,Credit,0.724862,0.0,1.0,1.268083
4,6838,G,male,Investment,-0.475651,2.0,0.0,1.204380
...,...,...,...,...,...,...,...,...
4995,1291,E,male,Credit,-0.263859,1.0,0.0,1.532543
4996,9280,G,male,Deposit,1.364917,3.0,0.0,5.256187
4997,4951,E,male,Deposit,0.894228,1.0,0.0,2.178767
4998,14569,E,male,Deposit,-0.447806,3.0,0.0,4.749217


In [87]:
data.df.columns

Index(['info_col_1', 'info_col_2', 'feature_col_1', 'feature_col_2',
       'feature_col_3', 'feature_col_4', 'treatment_1', 'outcome_1'],
      dtype='object')

In [88]:
data.df.treatment_1.value_counts()

treatment_1
0.0    2551
1.0    2449
Name: count, dtype: int64

In [89]:
data.df.isna().sum()

info_col_1         0
info_col_2         0
feature_col_1      0
feature_col_2    500
feature_col_3    500
feature_col_4      0
treatment_1        0
outcome_1          0
dtype: int64

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [90]:
info_col = data.info_col_names

outcome = data.outcome_name[0]
treatment = data.treatment_name[0]

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [91]:
# Standard model with base parameters
model = Matcher(input_data=data.df, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results, df_matched = model.estimate()

[23.11.2023 09:59:04 | hypex | INFO]: Number of NaN values filled with zeros: 1000
Get treated index: 100%|██████████| 5000/5000 [00:00<00:00, 47225.60it/s]


In [92]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,2.801526,0.066779,0.0,2.670639,2.932413,outcome_1
ATC,2.782048,0.075372,0.0,2.634318,2.929778,outcome_1
ATT,2.821815,0.074687,0.0,2.675429,2.968200,outcome_1


In [93]:
quality_results.keys()

dict_keys(['psi', 'ks_test', 'smd', 'repeats'])

In [94]:
quality_results['ks_test']

,match_control_to_treat,match_treat_to_control
feature_col_3,0.997195,0.999381
feature_col_4,1.000000,1.000000


In [95]:
df_matched

,index,feature_col_3,feature_col_4,feature_col_1_male,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,feature_col_3_matched,feature_col_4_matched,feature_col_1_male_matched,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,index_matched,outcome_1,outcome_1_matched,outcome_1_matched_bias,treatment_1,treatment_1_matched
0,1,0.764466,0.0,1,0,0,0,0.746784,0.0,1.0,0.0,0.0,0.0,[4391],1.510945,0.992153,0.500881,1,0
1,2,0.036796,0.0,0,0,1,0,0.042363,0.0,0.0,0.0,1.0,0.0,[354],1.392190,0.141016,1.256813,1,0
2,3,0.724862,0.0,1,1,0,0,0.711571,0.0,1.0,1.0,0.0,0.0,[4319],1.268083,0.382183,0.872439,1,0
3,5,0.040043,1.0,0,0,1,0,0.038750,1.0,0.0,0.0,1.0,0.0,[1702],4.740209,1.623045,3.115855,1,0
4,8,-1.513104,1.0,1,0,1,0,-1.153951,1.0,1.0,0.0,1.0,0.0,[1917],0.399753,-0.742346,1.505884,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,4993,-1.135007,0.0,0,0,1,0,-1.127268,0.0,0.0,0.0,1.0,0.0,[467],-1.108439,-3.100447,-2.007348,0,1
2547,4995,-0.263859,1.0,1,1,0,0,-0.251755,1.0,1.0,1.0,0.0,0.0,[2805],1.532543,1.161971,-0.394565,0,1
2548,4996,1.364917,3.0,1,0,1,0,1.371142,3.0,1.0,0.0,1.0,0.0,[1996],5.256187,11.577960,6.309433,0,1
2549,4997,0.894228,1.0,1,0,1,0,0.996776,1.0,1.0,0.0,1.0,0.0,[1506],2.178767,6.609450,4.227404,0,1


In [99]:
df_matched[df_matched['feature_col_1_male'] != df_matched['feature_col_1_male_matched']]

,index,feature_col_3,feature_col_4,feature_col_1_male,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,feature_col_3_matched,feature_col_4_matched,feature_col_1_male_matched,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,index_matched,outcome_1,outcome_1_matched,outcome_1_matched_bias,treatment_1,treatment_1_matched


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [100]:
group_col = 'feature_col_1'

In [101]:
model = Matcher(input_data=data.df, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)
results, quality_results, df_matched = model.estimate()

[23.11.2023 10:01:10 | hypex | INFO]: Number of NaN values filled with zeros: 1000
Get treated index by group male: 100%|██████████| 4/4 [00:00<00:00, 20.21it/s]  


In [102]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,2.801526,0.066779,0.0,2.670638,2.932413,outcome_1
ATC,2.782048,0.075372,0.0,2.634318,2.929778,outcome_1
ATT,2.821815,0.074687,0.0,2.675429,2.968201,outcome_1


In [103]:
df_matched

,index,feature_col_3,feature_col_4,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,feature_col_1,feature_col_3_matched,feature_col_4_matched,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,feature_col_1_matched,index_matched,outcome_1,outcome_1_matched,outcome_1_matched_bias,treatment_1,treatment_1_matched
0,2,0.036796,0.0,0,1,0,female,0.042363,0.0,0.0,1.0,0.0,female,[354],1.392190,0.141016,1.256813,1,0
1,5,0.040043,1.0,0,1,0,female,0.038750,1.0,0.0,1.0,0.0,female,[1702],4.740209,1.623045,3.115855,1,0
2,12,0.109878,1.0,0,0,1,female,0.108381,1.0,0.0,0.0,1.0,female,[2415],4.399381,2.384703,2.013162,1,0
3,14,-0.945215,0.0,0,1,0,female,-0.947401,0.0,0.0,1.0,0.0,female,[1109],-0.736954,-2.795847,2.056678,1,0
4,17,2.128255,1.0,0,1,0,female,2.011505,1.0,0.0,1.0,0.0,female,[3023],8.786283,3.595781,5.072241,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,4992,0.227277,0.0,1,0,0,male,0.201478,0.0,1.0,0.0,0.0,male,[2438],0.203098,1.538884,1.386938,0,1
2547,4995,-0.263859,1.0,1,0,0,male,-0.251755,1.0,1.0,0.0,0.0,male,[2805],1.532543,1.161971,-0.394571,0,1
2548,4996,1.364917,3.0,0,1,0,male,1.371142,3.0,0.0,1.0,0.0,male,[1996],5.256187,11.577960,6.309430,0,1
2549,4997,0.894228,1.0,0,1,0,male,0.996776,1.0,0.0,1.0,0.0,male,[1506],2.178767,6.609450,4.227354,0,1


In [104]:
df_matched[df_matched['feature_col_1'] != df_matched['feature_col_1_matched']]

,index,feature_col_3,feature_col_4,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,feature_col_1,feature_col_3_matched,feature_col_4_matched,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,feature_col_1_matched,index_matched,outcome_1,outcome_1_matched,outcome_1_matched_bias,treatment_1,treatment_1_matched


## 3. Results  
### 3.1 ATE, ATT, ATC

In [105]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,2.801526,0.066779,0.0,2.670638,2.932413,outcome_1
ATC,2.782048,0.075372,0.0,2.634318,2.929778,outcome_1
ATT,2.821815,0.074687,0.0,2.675429,2.968201,outcome_1


### 3.2 SMD, PSI, KS-test, repeats

In [106]:
quality_results.keys()

dict_keys(['psi', 'ks_test', 'smd', 'repeats'])

In [107]:
quality_results['psi']

,column_treated,anomaly_score_treated,check_result_treated,column_untreated,anomaly_score_untreated,check_result_untreated
0,feature_col_1_treated,0.0,OK,feature_col_1_untreated,0.0,OK
1,feature_col_2_Credit_treated,0.0,OK,feature_col_2_Credit_untreated,0.0,OK
2,feature_col_2_Deposit_treated,0.0,OK,feature_col_2_Deposit_untreated,0.0,OK
3,feature_col_2_Investment_treated,0.0,OK,feature_col_2_Investment_untreated,0.0,OK
4,feature_col_3_treated,0.0,OK,feature_col_3_untreated,0.0,OK
5,feature_col_4_treated,0.0,OK,feature_col_4_untreated,0.0,OK


In [108]:
quality_results['ks_test']

,match_control_to_treat,match_treat_to_control
feature_col_3,0.997195,0.999381
feature_col_4,1.000000,1.000000


In [109]:
quality_results['repeats']

{'match_control_to_treat': 0.31, 'match_treat_to_control': 0.28}

### 3.3 Validation
Validates estimated effect:
1. by replacing real treatment (`random_treatment`) with random placebo treatment.
 Estimated effect must be droped to zero;
2. by adding random feature (`random_feature`). Estimated effect shouldn't change
significantly, p-val < 0.05;
3. estimates effect on subset of data (`subset_refuter`) (default fraction is 0.8). Estimated effect
shouldn't change significantly, p-val < 0.05.

In [110]:
model.validate_result(refuter="random_treatment", effect_type="att", n_sim=10)

100%|██████████| 10/10 [00:06<00:00,  1.49it/s]


{'outcome_1': [0.0022408059967929917, 0.0]}

## 4. Save model

In [111]:
model.save("test_model.pickle")

In [112]:
model2 = Matcher.load("test_model.pickle")

In [113]:
model2.results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,2.801526,0.066779,0.0,2.670638,2.932413,outcome_1
ATC,2.782048,0.075372,0.0,2.634318,2.929778,outcome_1
ATT,2.821815,0.074687,0.0,2.675429,2.968201,outcome_1


In [114]:
model.results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,2.801526,0.066779,0.0,2.670638,2.932413,outcome_1
ATC,2.782048,0.075372,0.0,2.634318,2.929778,outcome_1
ATT,2.821815,0.074687,0.0,2.675429,2.968201,outcome_1
